In [85]:
# -*- coding: utf-8 -*-
''' 
@title ：基于大模型的搜索问答
@description ：结合搜索引擎和大模型，从网络搜索相关内容，总结搜索结果，并给出相关问题。
@author :Pantrick Zhang
@date 2025-01-20 10:09:01
@refence: https://github.com/shibing624/SearchGPT 
'''
import datetime
import json
import os
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional, Dict
from fastapi import FastAPI, HTTPException
from fastapi.responses import StreamingResponse, RedirectResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from loguru import logger
import uuid

class Language(Enum):
    """支持的语言枚举"""
    EN = "en"
    ZH = "zh"
    
class PromptManager:
    """提示词管理类"""

    def __init__(self):
        self._prompts = {
            "rag_system": {
                Language.EN: """You are a large language AI assistant. You are given a user question, and please write clean, concise and accurate answer to the question. You will be given a set of related contexts to the question, each starting with a reference number like [[citation:x]], where x is a number. Please use the context and cite the context at the end of each sentence if applicable.

Your answer must be correct, accurate and written by an expert using an unbiased and professional tone. Please keep your answer within 1024 tokens. If the provided context does not offer enough information, please use your own knowledge to answer the user question.

Please cite the contexts with the reference numbers, in the format [citation:x]. If a sentence comes from multiple contexts, please list all applicable citations, like [citation:3][citation:5]. Other than code and specific names and citations, your answer must be written in the same language as the question.
""",
                Language.ZH: """你是一个大型的语言AI助手。当用户提出问题时，请你写出清晰、简洁且准确的答案。我们会给你一组与问题相关的上下文，每个上下文都以类似[[citation:x]]这样的引用编号开始，其中x是一个数字。如果有引用[context]，请在每句话后面使用并引述该上下文。

你的答案必须正确、精确，并由专家以公正和专业的语气撰写。请将你的回答限制在1024个token内。如果所提供的上下文信息不足，可以使用自己知识来回答用户问题。

请按照[citation:x]格式引用带有参考编号的上下文。如果一句话来自多个上下文，请列出所有适用于此处引述，如[citation:3][citation:5]。除代码、特定名称和引述外，你必须使用与问题相同语言编写你的回答。
"""
            },
            "rag_qa": {
                Language.EN: """[context]=```
{context}
```
Current date: {current_date}

Please answer the question with contexts, but don't blindly repeat the contexts verbatim. Please cite the contexts with the reference numbers, in the format [citation:x]. And here is the user question:
""",
                Language.ZH: """[context]=```
{context}
```
当前日期: {current_date}

基于上下文回答问题，不要盲目地逐字重复上下文。请以[citation:x]的格式引用上下文。这是用户的问题：
"""
            },
            "related_system": {
                Language.EN: """You assist users in posing relevant questions based on their original queries and related background. Please identify topics worth following up on, and write out questions that each do not exceed 10 tokens. You Can combine with historical messages.""",
                Language.ZH: """你帮助用户根据他们的原始问题和相关背景提出相关问题，可以结合历史消息。请确定值得跟进的主题，每个问题不超过10个token。"""
            },
            "related_qa": {
                Language.EN: """[context]=```
{context}
```

based on the original question and related contexts, suggest three such further questions. Do NOT repeat the original question. Each related question should be no more than 10 tokens, separate 3 questions with `\\n`, do not add numbers before questions, just give question contents. Here is the original question:
""",
                Language.ZH: """[context]=```
{context}
```

根据原始问题和相关上下文，提出三个相似的问题。不要重复原始问题。每个相关问题应不超过10个token，问题前不要加序号，问题后加问号，`\\n`分隔多个问题。这是原始问题：
"""
            },
        }

    def get_prompt(self, prompt_type: str, lang: Language, **kwargs) -> str:
        """获取提示词
        
        Args:
            prompt_type: 提示词类型
            lang: 语言
            **kwargs: 格式化参数
        
        Returns:
            格式化后的提示词
        """
        prompt = self._prompts.get(prompt_type, {}).get(lang, "")
        if not prompt:
            raise ValueError(f"Prompt not found for type {prompt_type} and language {lang}")

        if kwargs:
            prompt = prompt.format(**kwargs)

        return prompt

    def detect_language(self, text: str) -> Language:
        """检测文本语言
        
        Args:
            text: 待检测文本
        
        Returns:
            检测到的语言
        """
        return Language.ZH if any('\u4e00' <= char <= '\u9fff' for char in text) else Language.EN


In [87]:
# step 1: 基于智谱AI提供的搜索工具搜索网络内容
import requests
import uuid
# https://open.bigmodel.cn/dev/api/search-tool/web-search-pro
def search_with_searchpro(query: str, api_key: str = None):
    try:
        api_key = api_key or os.environ.get("ZHIPUAI_API_KEY")
        if not api_key:
            raise RuntimeError("ZHIPUAI_API_KEY not set")
        logger.info(f"Searching web for: {query}")
        msg = [{"role": "user", "content": query}]
        data = {
            "request_id": str(uuid.uuid4()),
            "tool": "web-search-pro",
            "stream": False,
            "messages": msg
        }

        response = requests.post(
            url=SEARCHPRO_ENDPOINT,
            json=data,
            headers={'Authorization': api_key},
            timeout=120 # s
            )
        print(response)
        

        json_content = response.json()
        try:
            search_results = json_content['choices'][0]['message']['tool_calls'][1]['search_result']
            contexts = []

            for result in search_results[:REFERENCE_COUNT]:
                contexts.append({
                    "name": result.get("title", ""),
                    "url": result.get("link", ""),
                    "snippet": result.get("content", "")
                })
            logger.debug(f"Search results: {contexts}")
            return contexts
        except KeyError:
            logger.error(f"Error encountered: {json_content}")
            return []
    except Exception as e:
        logger.error(f"Failed to search with SearchPro: {e}")
        return []

SEARCHPRO_ENDPOINT = "https://open.bigmodel.cn/api/paas/v4/tools"

REFERENCE_COUNT = 10
DEFAULT_SEARCH_ENGINE_TIMEOUT = 5
ZHIPUAI_API_KEY='4cbbdb4079ce50378af38685e7927e3f.pF2jBgOkUji8KgKd'
api_key = ZHIPUAI_API_KEY
search_function = lambda q: search_with_searchpro(q, api_key)
q = '开源多模态大模型最新进展?'
contexts=search_function(q)    
for x in contexts:
    print(x)

2025-01-22 10:07:53.926 | INFO     | __main__:search_with_searchpro:12 - Searching web for: 开源多模态大模型最新进展?
2025-01-22 10:07:54.731 | DEBUG    | __main__:search_with_searchpro:41 - Search results: [{'name': '只有模仿没有创新？国产 AI 用行动打破质疑', 'url': 'https://finance.sina.com.cn/tech/roll/2025-01-17/doc-ineffyrh3466257.shtml', 'snippet': '摘要\nMiniMax，选择用开源「震撼」一下全球同行。\n作者  Li Yuan\n编辑  郑玄\n大模型时代已经正式迈入第三年。\n回首过去的两年，不禁令人感慨。每年都有基座大模型架构已经尘埃落定的声音，然而每年，技术都在飞快地迭代创新，突破人们想象。\n2024 年，OpenAI 的推理模型，通过对模型架构的创新，用 RL 的方法延续 Scaling Law，让大模型的智力水平持续进展；而中国公司也并没有落后，价格屠夫 DeepSeek 通过 MLA 的架构创新，让推理成本直接降低了一个数量级。\n2025 年开年，令人欣喜的是，我们看到了一向在人们印象中是「低调做产品」的 MiniMax 公司，也加入了开源行列，将最先进的底层技术直接与社区和行业分享。\n1 月 15 日，大模型公司 MiniMax 正式发布了 MiniMax-01 系列模型。它包括基础语言大模型 MiniMax-Text-01，和在其上集成了一个轻量级 ViT 模型而开发的视觉多模态大模型 MiniMax-VL-01。\n「卷」起来的大模型公司，令人乐见。开源会提升创新效率，越来越好的基座模型之上，才搭建越来越有用的应用，进入千家万户，帮人们解放生产力。\n这是 MiniMax 第一次发布开源模型，一出手就是一个炸裂模型架构创新：新模型采用了 MiniMax 独有的 Lightning Attention 机制，借鉴了 Linear Attention（线性注意力）机制，是全球第一次将 Linear Attention 机制引入到商业化规模的模型

<Response [200]>
{'name': '只有模仿没有创新？国产 AI 用行动打破质疑', 'url': 'https://finance.sina.com.cn/tech/roll/2025-01-17/doc-ineffyrh3466257.shtml', 'snippet': '摘要\nMiniMax，选择用开源「震撼」一下全球同行。\n作者  Li Yuan\n编辑  郑玄\n大模型时代已经正式迈入第三年。\n回首过去的两年，不禁令人感慨。每年都有基座大模型架构已经尘埃落定的声音，然而每年，技术都在飞快地迭代创新，突破人们想象。\n2024 年，OpenAI 的推理模型，通过对模型架构的创新，用 RL 的方法延续 Scaling Law，让大模型的智力水平持续进展；而中国公司也并没有落后，价格屠夫 DeepSeek 通过 MLA 的架构创新，让推理成本直接降低了一个数量级。\n2025 年开年，令人欣喜的是，我们看到了一向在人们印象中是「低调做产品」的 MiniMax 公司，也加入了开源行列，将最先进的底层技术直接与社区和行业分享。\n1 月 15 日，大模型公司 MiniMax 正式发布了 MiniMax-01 系列模型。它包括基础语言大模型 MiniMax-Text-01，和在其上集成了一个轻量级 ViT 模型而开发的视觉多模态大模型 MiniMax-VL-01。\n「卷」起来的大模型公司，令人乐见。开源会提升创新效率，越来越好的基座模型之上，才搭建越来越有用的应用，进入千家万户，帮人们解放生产力。\n这是 MiniMax 第一次发布开源模型，一出手就是一个炸裂模型架构创新：新模型采用了 MiniMax 独有的 Lightning Attention 机制，借鉴了 Linear Attention（线性注意力）机制，是全球第一次将 Linear Attention 机制引入到商业化规模的模型当中。\n效果也是立竿见影，模型上下文长度直接达到了顶尖模型的 20-32 倍水平，推理时的上下文窗口能达到 400 万 token。模型效果立刻在海外上引起了关注。\n模型的上下文窗口，指的是模型在生成每个新 token 时，实际参考的前面内容的范围。就像是模型能够一次性从书架上取下的书籍数量。模型的上下文窗口越大，模型生成时可以参考的信息量就越多，表现

In [89]:
'''
step2： 根据网络内容，使用智谱LLM生成总结，并带有[citation:9]引用标记
'''
from zhipuai import ZhipuAI
def generate_answer_zhipu(messages):
        client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
        response = client.chat.completions.create(
            model="glm-4-plus",  # 请填写您要调用的模型名称
            messages=messages,
            )
        response_text = response.choices[0].message.content          
        if response_text:
            history.append({"role": "assistant", "content": response_text})
            logger.debug(f"history: {history}")
        return response_text

def generate_answer(request) -> StreamingResponse:
        """LLM生成服务 - 返回生成的回答"""
        try:
            # request = json.loads(request)
            # 检测语言
            lang = pm.detect_language(request['query'])

            # 生成回答
            if not history:
                content = pm.get_prompt("rag_system", lang)
                history.append({"role": "system", "content": content})
            context = ""
            if request['contexts']:
                context = "\n\n".join(
                    [f"[[citation:{i + 1}]] {c.get('snippet', '')}" for i, c in enumerate(request['contexts'])])
            prompt = pm.get_prompt(
                "rag_qa",
                lang,
                context=context,
                current_date=datetime.datetime.today().strftime("%Y-%m-%d")
            )

            messages = history + [{"role": "user", "content": f"{prompt}\n\n{request['query']}"}]
            logger.debug(f"messages: {messages}")

            answer = generate_answer_zhipu(messages)
            return answer

        except Exception as e:
            logger.error(f"Error in generate: {e}")
            return JSONResponse(
                content={"error": str(e)},
                status_code=503
            )   

req = {'query':q, 'contexts':contexts}
pm = PromptManager()
history = []
print(req)            

generate_answer(req)            

2025-01-22 10:08:17.846 | DEBUG    | __main__:generate_answer:40 - messages: [{'role': 'system', 'content': '你是一个大型的语言AI助手。当用户提出问题时，请你写出清晰、简洁且准确的答案。我们会给你一组与问题相关的上下文，每个上下文都以类似[[citation:x]]这样的引用编号开始，其中x是一个数字。如果有引用[context]，请在每句话后面使用并引述该上下文。\n\n你的答案必须正确、精确，并由专家以公正和专业的语气撰写。请将你的回答限制在1024个token内。如果所提供的上下文信息不足，可以使用自己知识来回答用户问题。\n\n请按照[citation:x]格式引用带有参考编号的上下文。如果一句话来自多个上下文，请列出所有适用于此处引述，如[citation:3][citation:5]。除代码、特定名称和引述外，你必须使用与问题相同语言编写你的回答。\n'}, {'role': 'user', 'content': '[context]=```\n[[citation:1]] 摘要\nMiniMax，选择用开源「震撼」一下全球同行。\n作者  Li Yuan\n编辑  郑玄\n大模型时代已经正式迈入第三年。\n回首过去的两年，不禁令人感慨。每年都有基座大模型架构已经尘埃落定的声音，然而每年，技术都在飞快地迭代创新，突破人们想象。\n2024 年，OpenAI 的推理模型，通过对模型架构的创新，用 RL 的方法延续 Scaling Law，让大模型的智力水平持续进展；而中国公司也并没有落后，价格屠夫 DeepSeek 通过 MLA 的架构创新，让推理成本直接降低了一个数量级。\n2025 年开年，令人欣喜的是，我们看到了一向在人们印象中是「低调做产品」的 MiniMax 公司，也加入了开源行列，将最先进的底层技术直接与社区和行业分享。\n1 月 15 日，大模型公司 MiniMax 正式发布了 MiniMax-01 系列模型。它包括基础语言大模型 MiniMax-Text-01，和在其上集成了一个轻量级 ViT 模型而开发的视觉多模态大模型 MiniMax-VL-01。\n「卷」起来的大模型公司，令人乐见。开源会提升创新效率，越来越好的基座模型之上，

{'query': '开源多模态大模型最新进展?', 'contexts': [{'name': '只有模仿没有创新？国产 AI 用行动打破质疑', 'url': 'https://finance.sina.com.cn/tech/roll/2025-01-17/doc-ineffyrh3466257.shtml', 'snippet': '摘要\nMiniMax，选择用开源「震撼」一下全球同行。\n作者  Li Yuan\n编辑  郑玄\n大模型时代已经正式迈入第三年。\n回首过去的两年，不禁令人感慨。每年都有基座大模型架构已经尘埃落定的声音，然而每年，技术都在飞快地迭代创新，突破人们想象。\n2024 年，OpenAI 的推理模型，通过对模型架构的创新，用 RL 的方法延续 Scaling Law，让大模型的智力水平持续进展；而中国公司也并没有落后，价格屠夫 DeepSeek 通过 MLA 的架构创新，让推理成本直接降低了一个数量级。\n2025 年开年，令人欣喜的是，我们看到了一向在人们印象中是「低调做产品」的 MiniMax 公司，也加入了开源行列，将最先进的底层技术直接与社区和行业分享。\n1 月 15 日，大模型公司 MiniMax 正式发布了 MiniMax-01 系列模型。它包括基础语言大模型 MiniMax-Text-01，和在其上集成了一个轻量级 ViT 模型而开发的视觉多模态大模型 MiniMax-VL-01。\n「卷」起来的大模型公司，令人乐见。开源会提升创新效率，越来越好的基座模型之上，才搭建越来越有用的应用，进入千家万户，帮人们解放生产力。\n这是 MiniMax 第一次发布开源模型，一出手就是一个炸裂模型架构创新：新模型采用了 MiniMax 独有的 Lightning Attention 机制，借鉴了 Linear Attention（线性注意力）机制，是全球第一次将 Linear Attention 机制引入到商业化规模的模型当中。\n效果也是立竿见影，模型上下文长度直接达到了顶尖模型的 20-32 倍水平，推理时的上下文窗口能达到 400 万 token。模型效果立刻在海外上引起了关注。\n模型的上下文窗口，指的是模型在生成每个新 token 时，实际参考的前面内容的范围。就像是模型能够一次性从书架上取下的书籍数量。模型的上下文窗

2025-01-22 10:08:33.383 | DEBUG    | __main__:generate_answer_zhipu:14 - history: [{'role': 'system', 'content': '你是一个大型的语言AI助手。当用户提出问题时，请你写出清晰、简洁且准确的答案。我们会给你一组与问题相关的上下文，每个上下文都以类似[[citation:x]]这样的引用编号开始，其中x是一个数字。如果有引用[context]，请在每句话后面使用并引述该上下文。\n\n你的答案必须正确、精确，并由专家以公正和专业的语气撰写。请将你的回答限制在1024个token内。如果所提供的上下文信息不足，可以使用自己知识来回答用户问题。\n\n请按照[citation:x]格式引用带有参考编号的上下文。如果一句话来自多个上下文，请列出所有适用于此处引述，如[citation:3][citation:5]。除代码、特定名称和引述外，你必须使用与问题相同语言编写你的回答。\n'}, {'role': 'assistant', 'content': '开源多模态大模型的最新进展主要体现在以下几个方面：\n\n1. **MiniMax的开源创新**：MiniMax公司在2025年初发布了开源的MiniMax-01系列模型，包括基础语言大模型MiniMax-Text-01和视觉多模态大模型MiniMax-VL-01。这些模型采用了独有的Lightning Attention机制，借鉴了Linear Attention，显著提升了模型的上下文长度，达到顶尖模型的20-32倍，推理时的上下文窗口可达400万token，对Agent能力的发展具有重要意义[citation:1][citation:2][citation:3]。\n\n2. **多模态大语言模型（MLLM）的演进**：MLLM能够处理多模态任务，如视觉问答、图表推理等。其架构包括编码器、连接器和大语言模型，数据训练分为模态对齐和指令微调两阶段。模型演进体现在分辨率提高、输入形式丰富、I/O模态支持增加等方面。团队工作包括缓解幻觉、利用外部反馈提升准确性、构建长视频理解测评基准等[citation:4]。\n\n3. **字节跳动的豆包实时语音大模型**：字节跳动发布了豆包实时语音大模型，实现了端到端语音对话

'开源多模态大模型的最新进展主要体现在以下几个方面：\n\n1. **MiniMax的开源创新**：MiniMax公司在2025年初发布了开源的MiniMax-01系列模型，包括基础语言大模型MiniMax-Text-01和视觉多模态大模型MiniMax-VL-01。这些模型采用了独有的Lightning Attention机制，借鉴了Linear Attention，显著提升了模型的上下文长度，达到顶尖模型的20-32倍，推理时的上下文窗口可达400万token，对Agent能力的发展具有重要意义[citation:1][citation:2][citation:3]。\n\n2. **多模态大语言模型（MLLM）的演进**：MLLM能够处理多模态任务，如视觉问答、图表推理等。其架构包括编码器、连接器和大语言模型，数据训练分为模态对齐和指令微调两阶段。模型演进体现在分辨率提高、输入形式丰富、I/O模态支持增加等方面。团队工作包括缓解幻觉、利用外部反馈提升准确性、构建长视频理解测评基准等[citation:4]。\n\n3. **字节跳动的豆包实时语音大模型**：字节跳动发布了豆包实时语音大模型，实现了端到端语音对话，具备低时延、对话中可随时打断等特性。该模型在情绪理解和情感表达方面表现优异，支持多模态输入输出，涵盖S2S、S2T、T2S、T2T等多种模式[citation:6]。\n\n4. **Meta的Spirit LM**：Meta开源了7B参数的Spirit LM多模态语言模型，能够理解和生成语音及文本，自然地在两种模式间转换。该模型不仅处理基本的语音转文本和文本转语音任务，还能捕捉和再现语音中的情感和风格[citation:8]。\n\n5. **腾讯的HunYuan-Video模型**：腾讯混元大模型上线了视频生成能力，通过引入超大规模数据处理系统、多模态大语言模型、全注意力机制和自研3D VAE架构，提升了视频生成的质量和可控性[citation:9]。\n\n这些进展表明，开源多模态大模型在技术架构、应用场景和性能提升方面都取得了显著成果，为AI的进一步发展和应用奠定了坚实基础。'

In [91]:
'''
step3： 使用智谱LLM生成相关问题。
'''
related_history =[]

def get_related_questions(query: str, contexts: List[dict]) -> List[str]:
        """获取相关问题"""
        try:
            lang = pm.detect_language(query)
            context = ""
            if contexts:
                context = "\n\n".join([c["snippet"][:50] for c in contexts[:3]])
            qa_prompt = pm.get_prompt(
                "related_qa",
                lang,
                context=context
            )
            user_prompt = f"{qa_prompt}\n\n{query}"
            logger.debug(f"related prompt: {user_prompt}")
            
            client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
            response = client.chat.completions.create(
            model="glm-4-plus",  # 请填写您要调用的模型名称
            messages= [{"role": "user", "content": user_prompt}],
            max_tokens=512,
            )
                       
            related_history.append({"role": "user", "content": query})            

            # 解析回答生成相关问题
            answer = response.choices[0].message.content
            # \n分隔，或者？或者?分隔
            questions = [q.strip() for q in answer.split('\n') if q.strip()]
            if len(questions) == 1:
                questions = [q + '？' for q in answer.split('？') if q.strip()]
            if len(questions) == 1:
                questions = [q + '?' for q in answer.split('?') if q.strip()]
            r = questions[:5]
            logger.debug(f"Related questions: {r}")
            return r
        except Exception as e:
            logger.error(f"Error in generating related questions: {e}")
            return []

def get_related( request):
        """相关问题生成服务"""
        try:
            # request = json.loads(request)
            # related_history = []

            if not related_history:
                content = pm.get_prompt(
                    "related_system",
                    pm.detect_language(request['query'])
                )
                related_history.append({"role": "system", "content": content})

            questions = get_related_questions(request['query'], request['contexts'])
            return JSONResponse(content=questions)
        except Exception as e:
            logger.error(f"Error in related: {e}")
            return JSONResponse(
                content={"error": str(e)},
                status_code=503
            )

releated_content = get_related(req)                    

2025-01-22 10:08:38.993 | DEBUG    | __main__:get_related_questions:16 - related prompt: [context]=```
摘要
MiniMax，选择用开源「震撼」一下全球同行。
作者  Li Yuan
编辑  郑玄
大模型

MiniMax，选择用开源「震撼」一下全球同行。
大模型时代已经正式迈入第三年。
回首过去的两年，不

「2025 年，我们可能会看到第一批 AI Agent 加入劳动力大军，并对公司的生产力产生实质性的
```

根据原始问题和相关上下文，提出三个相似的问题。不要重复原始问题。每个相关问题应不超过10个token，问题前不要加序号，问题后加问号，`\n`分隔多个问题。这是原始问题：


开源多模态大模型最新进展?
2025-01-22 10:08:40.178 | DEBUG    | __main__:get_related_questions:36 - Related questions: ['MiniMax开源策略影响?', '大模型时代趋势?', 'AI Agent 劳动力前景?']
